#  BM20A6100 Advanced Data Analysis and Machine Learning
## Erik Kuitunen, 0537275

Import packages

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import nltk
import wordcloud
import string

from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras.models import Sequential

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

2024-11-16 21:45:17.922184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731786317.956813  434797 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731786317.967010  434797 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 21:45:18.000239: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Read and preprocess text data

In [2]:
file = open("Grimm Tales.txt", 'rb')
lines = []
for line in file:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    
    if len(line) == 0:
        continue
    lines.append(line)
    
file.close()
text = " ".join(lines)

# set of characters that occur in the text
chars = set( [c for c in text] )

# Total items in our vocabulary
unique_chars = len( chars )

# lookup tables to deal with indexes of characters rather than the characters themselves.
char2index = dict( (c, i) for i, c in enumerate( chars ) )
index2char = dict( (i, c) for i, c in enumerate( chars ) )

Reshaping and one-hot encoding of the data

In [3]:
sequence_length = 10
step = 1
input_chars = []
label_chars = []
for i in range( 0, len(text) - sequence_length, step ):
    input_chars.append(text[i:i + sequence_length])
    label_chars.append(text[i + sequence_length])
    
X = np.zeros((len(input_chars), sequence_length, unique_chars), dtype=np.bool)
y = np.zeros((len(input_chars), unique_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
        

Model definition

In [4]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 50

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False, 
                    input_shape=(sequence_length, unique_chars), 
                    unroll=True))
model.add(Dense(unique_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

2024-11-16 21:45:23.422313: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/eki/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training and testing model

In [5]:
for iteration in range(NUM_ITERATIONS):
    print("Iteration #: %d" % (iteration))
    
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    
    print("\nGenerating from seed: %s" % (test_chars))
    print(test_chars, end="")
    
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros( (1, sequence_length, unique_chars) )
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="\n")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
# print()

Iteration #: 0
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - loss: 2.1846

Generating from seed: t, and her
t, and her and said the sood the sood the sood the sood the Iteration #: 1
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - loss: 1.7435

Generating from seed:  gift in r
 gift in round the wind was the princess and said the said tIteration #: 2
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - loss: 1.6021

Generating from seed: n. gretel 
n. gretel on the fire and the should not be the should not bIteration #: 3
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 36s 9ms/step - loss: 1.5230

Generating from seed: t, fell as
t, fell as the word was a little was stood a little was stooIteration #: 4
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - loss: 1.4788

Generating from seed:  i could b
 i could be a great down and said the stars and said the staIteration #: 5
4011/4011 ━━━━━━━━━━━━━━━━━━━━ 38s 10ms/step - loss: 1.4399

Generating from seed:  as she sa
 as she said the man that was a sad and said, i wil